# Read the data

In [2]:
import pandas as pd
# Read training & test data from csv 
df_train = pd.read_csv('buyComputer.csv')
df_train['cls'] = 'train'
df_test = pd.read_csv('buyComputer_test.csv')
df_test['cls'] = 'test'
# ==================================================================================================

df_data = pd.concat([ df_train, df_test ])
print (df_data)

   agelevel  income student     credit BuyComputer    cls
0     youth    high      no       fair          no  train
1     youth    high      no  excellent          no  train
2    middle    high      no       fair         yes  train
3    senior  medium      no       fair         yes  train
4    senior     low     yes       fair         yes  train
5    senior     low     yes  excellent          no  train
6    middle     low     yes  excellent         yes  train
7     youth  medium      no       fair          no  train
8     youth     low     yes       fair         yes  train
9    senior  medium     yes       fair         yes  train
10    youth  medium     yes  excellent         yes  train
11   middle  medium      no  excellent         yes  train
12   middle    high     yes       fair         yes  train
13   senior  medium      no  excellent          no  train
0     youth  medium     yes       fair          no   test


# Pre-process the data

In [3]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()


df_data['agelevel'] = df_data['agelevel'].map({ "youth": 1, "middle": 2, "senior": 3 }.get)
df_data['income'] = df_data['income'].map({ "low": 1, "medium": 2, "high": 3 }.get)
df_data['student'] = df_data['student'].map({ "no": 0, "yes": 1 }.get)
df_data['credit'] = df_data['credit'].map({ "fair": 0, "excellent": 1 }.get)
df_data['BuyComputer'] = df_data['BuyComputer'].map({ "no": 0, "yes": 1 }.get)

df_data

,agelevel,income,student,credit,BuyComputer,cls
0,1,3,0,0,0,train
1,1,3,0,1,0,train
2,2,3,0,0,1,train
3,3,2,0,0,1,train
4,3,1,1,0,1,train
5,3,1,1,1,0,train
6,2,1,1,1,1,train
7,1,2,0,0,0,train
8,1,1,1,0,1,train
9,3,2,1,0,1,train


# Split into train / test Data Sets

In [13]:
df_train = df_data[df_data['cls'] == 'train'].drop(['cls'], axis = 1)
df_test = df_data[df_data['cls'] == 'test'].drop(['cls'], axis = 1)

y_train = df_train['BuyComputer']
X_train = df_train.drop(['BuyComputer'], axis = 1)

y_test = df_test['BuyComputer']
X_test = df_test.drop(['BuyComputer'], axis = 1)

print('訓練資料(X)')
print('----------------------------------')
print(X_train)
print('訓練資料(y)')
print('----------')
print(y_train)
print('====================================================')
print('測試資料(X)')
print('----------------------------------')
print(X_test)
print('測試資料(y)')
print('----------')
print(y_test)

訓練資料(X)
----------------------------------
    agelevel  income  student  credit
0          1       3        0       0
1          1       3        0       1
2          2       3        0       0
3          3       2        0       0
4          3       1        1       0
5          3       1        1       1
6          2       1        1       1
7          1       2        0       0
8          1       1        1       0
9          3       2        1       0
10         1       2        1       1
11         2       2        0       1
12         2       3        1       0
13         3       2        0       1
訓練資料(y)
----------
0     0
1     0
2     1
3     1
4     1
5     0
6     1
7     0
8     1
9     1
10    1
11    1
12    1
13    0
Name: BuyComputer, dtype: int64
測試資料(X)
----------------------------------
   agelevel  income  student  credit
0         1       2        1       0
測試資料(y)
----------
0    0
Name: BuyComputer, dtype: int64


# Standardizing the data

In [18]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Training Data & Predict

In [19]:
# 開始訓練
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=0)
model.fit(X_train, y_train)

# 預測
y_pred = model.predict(X_test)
print('Misclassified samples: %d' % (y_test != y_pred).sum())
y_pred

# 預測結果  1: 代表會買; 0: 代表不會買
from sklearn.metrics import accuracy_score
print('測試資料的準確率: %.2f' % accuracy_score(y_test, y_pred))

Misclassified samples: 1
測試資料的準確率: 0.00


# 畫圖

到 http://www.graphviz.org/download/ 下載並安裝 graphviz 即可 

In [20]:
# 產生 tree.dot 檔
from sklearn.tree import export_graphviz
A=['no', 'yes']
export_graphviz(model, 
                out_file='tree.dot', 
                feature_names=['agelevel', 'income', 'student', 'credit'], 
                class_names=A, 
                rounded = True, 
                proportion =False, precision = 2, filled = True)

In [88]:
#!pip install graphviz

產生 tree.dot 檔後，開啟「命令提示字元」
輸入以下命令，產生 tree.png  

``` 
dot -Tpng .\tree.dot -o tree.png
```